In [9]:
pip install catboost

  Using cached catboost-1.2.7-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB 1.4 MB/s eta 0:01:15
   ---------------------------------------- 0.0/101.7 MB 1.4 MB/s eta 0:01:15
   ---------------------------------------- 0.0/101.7 MB 281.8 kB/s eta 0:06:01
   ---------------------------------------- 0.1/101.7 MB 365.7 kB/s eta 0:04:38
   ---------------------------------------- 0.1/101.7 MB 365.7 kB/s eta 0:04:38
   ------------------------------------

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [13]:
train_dir = r'E:\FCI\fourth level\computer vision\Alzheimer_s Dataset\train'  
test_dir = r'E:\FCI\fourth level\computer vision\Alzheimer_s Dataset\test'

In [15]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [17]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [19]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [21]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [23]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

E:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

E:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 75s 463ms/step


In [27]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 17s 429ms/step


In [48]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3240593	total: 34.6ms	remaining: 34.6s
1:	learn: 1.2742083	total: 55.2ms	remaining: 27.5s
2:	learn: 1.2298300	total: 75.5ms	remaining: 25.1s
3:	learn: 1.1920827	total: 93.9ms	remaining: 23.4s
4:	learn: 1.1594676	total: 112ms	remaining: 22.3s
5:	learn: 1.1305877	total: 131ms	remaining: 21.6s
6:	learn: 1.1063024	total: 149ms	remaining: 21.1s
7:	learn: 1.0850539	total: 167ms	remaining: 20.7s
8:	learn: 1.0649661	total: 186ms	remaining: 20.5s
9:	learn: 1.0477846	total: 208ms	remaining: 20.6s
10:	learn: 1.0308988	total: 230ms	remaining: 20.7s
11:	learn: 1.0148825	total: 254ms	remaining: 20.9s
12:	learn: 1.0017691	total: 279ms	remaining: 21.2s
13:	learn: 0.9897472	total: 299ms	remaining: 21.1s
14:	learn: 0.9788589	total: 319ms	remaining: 20.9s
15:	learn: 0.9692622	total: 337ms	remaining: 20.8s
16:	learn: 0.9582660	total: 356ms	remaining: 20.6s
17:	learn: 0.9481728	total: 374ms	remaining: 20.4s
18:	learn: 0.9397951	total: 392ms	remaining: 20.3s
19:	learn: 0.9325142	total: 411ms	rem

In [50]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [52]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 56.53%


In [54]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [56]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [60]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'E:\FCI\fourth level\computer vision\Alzheimer_s Dataset\test\VeryMildDemented\26 (45).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
The image is classified as: VeryMildDemented


C:\Users\Ram\AppData\Local\Temp\ipykernel_4552\3798677812.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
